Produce a plot of a gps track and find the best mile long speeds in the track.

The plot encodes the elevation in the size of the plot character (e.g., circle) and the speed is encoded in the color of the track. 

The two "best mile" speeds printed are for 
1. any elevation change (downhill or uphill)
2. any mile with 5% or greater grade


In [1]:
#handle imports, requires gpxpy, matplotlib, and numpy

#standard python imports
import os, sys, math
from os import listdir
from os.path import isfile, join
import glob

#add the gpxpy lib to the path
lib_path = os.path.abspath(os.path.join('..','gpxpy'))
sys.path.append(lib_path)
import gpxpy
import gpxpy.gpx
import gpxpy.utils as mod_utils
import gpxpy.geo as mod_geo
import matplotlib.pyplot as plt
import numpy as np

#gpxpy works in meters, since we want miles we have a constant
CONST_MILE_IN_METERS = 1609.34

In [17]:
#supporting functions

def format_time(time_s):
    if not time_s:
        return 'n/a'
    minutes = math.floor(time_s / 60.)
    hours = math.floor(minutes / 60.)

    return '%s:%s:%s' % (str(int(hours)).zfill(2), str(int(minutes % 60)).zfill(2), str(int(time_s % 60)).zfill(2))


def print_gpx_part_info(gpx_part, indentation='    '):
    """
    gpx_part may be a track or segment.
    """
    length_2d = gpx_part.length_2d()
    length_3d = gpx_part.length_3d()
    print('{}Length 2D: {:.3f}km'.format(indentation, length_2d / 1000.))
    print('{}Length 3D: {:.3f}km'.format(indentation, length_3d / 1000.))

    moving_time, stopped_time, moving_distance, stopped_distance, max_speed = gpx_part.get_moving_data()
    print('%sMoving time: %s' % (indentation, format_time(moving_time)))
    print('%sStopped time: %s' % (indentation, format_time(stopped_time)))
    print('{}Max speed: {:.2f}m/s = {:.2f}km/h'.format(indentation, max_speed if max_speed else 0, max_speed * 60. ** 2 / 1000. if max_speed else 0))

    uphill, downhill = gpx_part.get_uphill_downhill()
    print('{}Total uphill: {:.2f}m'.format(indentation, uphill))
    print('{}Total downhill: {:.2f}m'.format(indentation, downhill))

    start_time, end_time = gpx_part.get_time_bounds()
    print('%sStarted: %s' % (indentation, start_time))
    print('%sEnded: %s' % (indentation, end_time))

    points_no = len(list(gpx_part.walk(only_points=True)))
    print('%sPoints: %s' % (indentation, points_no))

    distances = []
    previous_point = None
    for point in gpx_part.walk(only_points=True):
        if previous_point:
            distance = point.distance_2d(previous_point)
            distances.append(distance)
        previous_point = point
    print('{}Avg distance between points: {:.2f}m'.format(indentation, sum(distances) / len(list(gpx_part.walk()))))

    print('')


def print_gpx_info(gpx, gpx_file):
    print('File: %s' % gpx_file)

    if gpx.name:
        print('  GPX name: %s' % gpx.name)
    if gpx.description:
        print('  GPX description: %s' % gpx.description)
    if gpx.author:
        print('  Author: %s' % gpx.author)
    if gpx.email:
        print('  Email: %s' % gpx.email)

    print_gpx_part_info(gpx)

    for track_no, track in enumerate(gpx.tracks):
        for segment_no, segment in enumerate(track.segments):
            print('    Track #%s, Segment #%s' % (track_no, segment_no))
            print_gpx_part_info(segment, indentation='        ')

#summarize the track, generate arrays of time & distance            
def get_time_and_distances(gpx, simplify = False):
    indentation='    '
    time_and_distances = []
    stopped_speed_threshold = 1
    for track in gpx.tracks:
        for segment in track.segments:
            if simplify:
                segment.smooth()
                segment.simplify()

            moving_time, stopped_time, moving_distance, stopped_distance, max_speed = segment.get_moving_data()
            print('%sMoving time: %s' % (indentation, format_time(moving_time)))
            print('%sStopped time: %s' % (indentation, format_time(stopped_time)))
            print('{}Max speed: {:.2f}m/s = {:.2f}km/h'.format(indentation, max_speed if max_speed else 0, max_speed * 60. ** 2 / 1000. if max_speed else 0))

            uphill, downhill = segment.get_uphill_downhill()
            print('{}Total uphill: {:.2f}m'.format(indentation, uphill))
            print('{}Total downhill: {:.2f}m'.format(indentation, downhill))

            for i in range(1, len(segment.points)):
                previous = segment.points[i-1]
                point = segment.points[i]

                first_or_last = i in [0, 1, len(segment.points) -1]
                if first_or_last:
                    continue
                if point.time and previous.time:
                    timedelta = point.time - previous.time

                    if point.elevation and previous.elevation:
                        distance = point.distance_3d(previous)
                    else: 
                        distance = point.distance_2d(previous)

                    seconds = mod_utils.total_seconds(timedelta)
                    speed_kmh = 0
                    if seconds > 0:
                        speed_kmh = (distance / 1000.) / (mod_utils.total_seconds(timedelta) / 60. ** 2)

                    if speed_kmh > stopped_speed_threshold:
                        moving_time = mod_utils.total_seconds(timedelta)

                        if distance and moving_time:
                            segment_subset = [segment.points[sub].elevation for sub in range(i-1, i+2)]
                            #segment_subset = segment.points[i-1:i+2:1]
                            #if i-1 < 3:
                            #    print '{0} to {1} is segment_subset: {2}'.format(i-1, i+1, segment_subset)
                            #
                            time_and_distances.append((seconds, distance, mod_geo.calculate_uphill_downhill(segment_subset),))

        return time_and_distances   

#get the fastest miles at any elevation change and at elevation change greater than threshold (meters) / distance
def find_fastest_distance(time_and_distances, distance_threshold = CONST_MILE_IN_METERS, elevation_threshold = 0):
    fastest_dist = 3600
    fastest_flat = 3600
    fastest_dist_elevation = 0
    fastest_dist_index = 0
    fastest_flat_elevation = 0
    fastest_flat_index = 0
    cumulative_distance = 0
    for index, item in enumerate(time_and_distances):
        current_distance = 0
        elapsed_seconds = 0
        elevation_change = 0
        cumulative_distance += item[1]
        for next_item in range(index, len(time_and_distances)):
            elapsed_seconds += time_and_distances[next_item][0]
            current_distance += time_and_distances[next_item][1]
            elevation_change += time_and_distances[next_item][2][0]-time_and_distances[next_item][2][1]

            # if distance is greater than 1 mile in meters check if we have a new fastest mile
            if current_distance > distance_threshold:
                if elapsed_seconds < fastest_dist:
                    fastest_dist = elapsed_seconds
                    fastest_dist_elevation = elevation_change
                    fastest_dist_index = cumulative_distance / CONST_MILE_IN_METERS
                # +-50 meters in elevation change over a 1609 meters is roughly 3% grade
                if elevation_change >= elevation_threshold:
                    if elapsed_seconds < fastest_flat:
                        fastest_flat = elapsed_seconds
                        fastest_flat_elevation = elevation_change
                        fastest_flat_index = cumulative_distance / CONST_MILE_IN_METERS
                break

    print 'Fastest mile any elevation: {0:.1f} mph starting at mile {1:.1f} with elevation change of {2:.1f}% in {3} s'.format((distance_threshold / CONST_MILE_IN_METERS) /(fastest_dist/3600.), fastest_dist_index, fastest_dist_elevation/CONST_MILE_IN_METERS*100, fastest_dist)
    print 'Fastest mile of at least {0:.1f}%: {1:.1f} mph starting at mile {2:.1f} with elevation change of {3:.1f}% in {4} s'.format(elevation_threshold / CONST_MILE_IN_METERS *100, (distance_threshold / CONST_MILE_IN_METERS) /(fastest_flat/3600.), fastest_flat_index, fastest_flat_elevation/CONST_MILE_IN_METERS*100, fastest_flat)

# plot the track and save the file in the same folder as the track
def plot_tracks(data, simplify=False):
    for activity_idx, activity in enumerate(data):
        lat = []
        lon = []
        ele = []
        spd = []

        fig = plt.figure(facecolor = '1')
        ax = plt.Axes(fig, [0., 0., 1., 1.], )
        ax.set_aspect('equal')
        ax.set_axis_off()
        fig.add_axes(ax)

        gpx_filename = activity
        gpx_file = open(gpx_filename, 'r')
        gpx = gpxpy.parse(gpx_file)

        for track in gpx.tracks:
            for segment in track.segments:
                if simplify:
                    segment.smooth()
                    #segment.simplify()

                previous = segment.points[0]
                for point in segment.points:
                    lat.append(point.latitude)
                    lon.append(point.longitude)
                    if point.elevation < 1:
                        ele.append(1)
                    else:
                        ele.append(point.elevation)

                    if point.time and previous.time:
                        timedelta = point.time - previous.time

                        if point.elevation and previous.elevation:
                            distance = point.distance_3d(previous)
                        else: 
                            distance = point.distance_2d(previous)

                        seconds = mod_utils.total_seconds(timedelta)
                        speed_kmh = 0
                        if seconds > 0:
                            speed_kmh = (distance / 1000.) / (mod_utils.total_seconds(timedelta) / 60. ** 2)
                        else:
                            speed_kmh = 1

                        if speed_kmh < 1:
                            speed_kmh = 1

                        if speed_kmh > 75:
                            speed_kmh = 75
                    spd.append(speed_kmh)
                    previous = point

        #consider scaling or thresholding speed & elevation if plotting multiple tracks
        spd = np.array(spd)
        #print(spd.max())
        #spd *= 50/spd.max()
        #print(spd)
        #print(spd.max())
        ele = np.array(ele)
        #print(ele.min())
        #print(ele.max())
        #print(ele)
        #ele *= 255/ele.max()
        #ele *= ele

        #various ways to plot 1+ tracks
        # 'deepskyblue' works well on black facecolor()
        #plt.plot(lon, lat, color = 'deepskyblue', lw = ele, alpha = 0.8)
        plt.scatter(lon, lat, s=ele, c=spd, cmap=plt.cm.hot, lw=0.05, alpha = 0.8)
        #plt.scatter(lon, lat, c=spd, cmap=plt.cm.cool, alpha = 0.01)

        plt.colorbar()
        filename = activity+'.png'
        plt.savefig(filename, facecolor = fig.get_facecolor(), bbox_inches='tight', pad_inches=0, dpi=300)

def get_best_performances(gpx_files):
    for gpx_file_path in gpx_files:
        gpx_file = open(gpx_file_path, 'r')
        gpx = gpxpy.parse(gpx_file)
        print(gpx_file_path)
        gpx_times = get_time_and_distances(gpx, simplify=True)
        distance_to_evaluate = CONST_MILE_IN_METERS
        minimum_elevation_change = 80 #elevation change in meters for best uphill mile
        find_fastest_distance(gpx_times, distance_to_evaluate, minimum_elevation_change)


Using a sample gpx file, generate the plot and the performance data. 

In [18]:
data_path = 'sample_tracks'
simplify = True

data = glob.glob(os.path.join(data_path+"/sample.gpx"))

get_best_performances(data)
plot_tracks(data, simplify)

sample_tracks/sample.gpx
    Moving time: 03:32:37
    Stopped time: 00:28:58
    Max speed: 12.04m/s = 43.34km/h
    Total uphill: 1014.60m
    Total downhill: 1013.60m
Fastest mile any elevation: 25.2 mph starting at mile 24.6 with elevation change of -8.4% m in 143 s
Fastest mile of at least 5.0%: 10.1 mph starting at mile 10.2 with elevation change of 5.1% m in 357 s


<img src='sample_tracks/sample.gpx.png'>